# **\[Essays\]** 강화학습(1) – bandit

신록예찬  
2023-08-23

## imports

In [173]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import torch
import collections
import IPython

# Intro

`-` 강화학습(대충설명): 어떠한 “(게임)환경”이 있을때 거기서 “뭘 할지”를
학습하는 과업

# Game1: 벤딧문제

`-` 문제설명: 두개의 버튼이 있다. 버튼0을 누르면 1의 보상을, 버튼1를
누르면 100의 보상을 준다고 가정

`-` 어떤 행동을 해야할까? –\> ?? 아는게없음 –\> 일단 “아무거나” 눌러보자

In [2]:
action_space = ['button0','button1']
action = np.random.choice(action_space)
action

`-` 보상은 아래와 같은 방식으로 받을 것이다.

In [3]:
if action == 'button0': 
    reward = 1 
else:
    reward = 100

`-` 아무거나 10번 버튼을 눌러보면 다음과 같은 결과가 나온다.

In [4]:
for _ in range(10):
    action = np.random.choice(action_space)
    if action == 'button0': 
        reward = 1 
    else:
        reward = 100
    print(action,reward)

button0 1
button0 1
button1 100
button1 100
button0 1
button0 1
button1 100
button1 100
button0 1
button0 1

`-` 깨달았음: `button0`을 누르면 1점을 받고, `button1`을 누르면 100점을
받는 “환경”이구나? $\to$ `button1`을 누르는 “동작”을 해야하는
상황이구나?

-   여기에서 $\to$ 의 과정을 체계화 시킨 학문이 강화학습이다.

In [5]:
action_space = ['button0','button1']
for _ in range(10):
    #action = np.random.choice(action_space) # 무지한자의 행동 (찍어)
    action = action_space[1] # 깨달은자의 행동 (button1을 눌러)
    if action == 'button0': 
        reward = 1 
    else:
        reward = 100
    print(action,reward)

button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100

`-` 강화학습: 환경의 이해 $\to$ 뭘 해야 하는지 깨달음

**위의 과정이 잘 되었다는 의미로 사용하는 문장들**

-   강화학습이 성공적으로 잘 되었다.
-   환경이 해결되었다.
-   에이전트가 환경의 과제를 완료했다.
-   에이전트가 환경에서 성공적으로 학습했다.
-   에이전트가 올바른 행동을 학습했다.
-   게임 클리어 (비공식)

`-` 환경이 해결되었는지 나타내는 지표를 정해야겠다 $\to$ 주어진 상황을
게임처럼 이해하고, 게임의 클리어조건을 설정

-   첫 생각: `button1`을 누르면 클리어 아니야?
-   두번째 생각: 아니지? 우연히 누를수도 있잖아.
-   게임클리어조건: 최근 20번의 보상이 1900 이상이면 게임이 클리어
    되었다고 보자![1]

`-` 무지한자 – 게임을 클리어 할 수 없다.

[1] `button1`을 눌러야하는건 맞지만 20번에 실수한번정도는 눈감아 주자

In [6]:
action_space = ['button0','button1']
rewards = []
for i in range(50): # 1000번해도 못깸
    action = np.random.choice(action_space) # 무지한자의 행동 (찍어)
    #action = action_space[1] # 깨달은자의 행동 (button1을 눌러)
    if action == 'button0': 
        reward = 1 
        rewards.append(reward)
    else:
        reward = 100
        rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break
    

n_try = 1   action = button1    reward = 100    rwd20 = 100
n_try = 2   action = button1    reward = 100    rwd20 = 200
n_try = 3   action = button0    reward = 1  rwd20 = 201
n_try = 4   action = button0    reward = 1  rwd20 = 202
n_try = 5   action = button0    reward = 1  rwd20 = 203
n_try = 6   action = button0    reward = 1  rwd20 = 204
n_try = 7   action = button0    reward = 1  rwd20 = 205
n_try = 8   action = button0    reward = 1  rwd20 = 206
n_try = 9   action = button0    reward = 1  rwd20 = 207
n_try = 10  action = button0    reward = 1  rwd20 = 208
n_try = 11  action = button1    reward = 100    rwd20 = 308
n_try = 12  action = button0    reward = 1  rwd20 = 309
n_try = 13  action = button1    reward = 100    rwd20 = 409
n_try = 14  action = button0    reward = 1  rwd20 = 410
n_try = 15  action = button1    reward = 100    rwd20 = 510
n_try = 16  action = button1    reward = 100    rwd20 = 610
n_try = 17  action = button1    reward = 100    rwd20 = 710
n_try = 18  action =

`-` 깨달은자 – 게임 클리어

In [8]:
action_space = ['button0','button1']
rewards = []
for i in range(50): # 1000번해도 못깸
    #action = np.random.choice(action_space) # 무지한자의 행동 (찍어)
    action = action_space[1] # 깨달은자의 행동 (button1을 눌러)
    if action == 'button0': 
        reward = 1 
        rewards.append(reward)
    else:
        reward = 100
        rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break

n_try = 1   action = button1    reward = 100    rwd20 = 100
n_try = 2   action = button1    reward = 100    rwd20 = 200
n_try = 3   action = button1    reward = 100    rwd20 = 300
n_try = 4   action = button1    reward = 100    rwd20 = 400
n_try = 5   action = button1    reward = 100    rwd20 = 500
n_try = 6   action = button1    reward = 100    rwd20 = 600
n_try = 7   action = button1    reward = 100    rwd20 = 700
n_try = 8   action = button1    reward = 100    rwd20 = 800
n_try = 9   action = button1    reward = 100    rwd20 = 900
n_try = 10  action = button1    reward = 100    rwd20 = 1000
n_try = 11  action = button1    reward = 100    rwd20 = 1100
n_try = 12  action = button1    reward = 100    rwd20 = 1200
n_try = 13  action = button1    reward = 100    rwd20 = 1300
n_try = 14  action = button1    reward = 100    rwd20 = 1400
n_try = 15  action = button1    reward = 100    rwd20 = 1500
n_try = 16  action = button1    reward = 100    rwd20 = 1600
n_try = 17  action = button1    r

# Game1: 벤딧문제 – 예쁜코드

## 수정1: `action_space`의 수정

In [9]:
action_space =  gym.spaces.Discrete(2)
action_space

In [10]:
for _ in range(10):
    print(action_space.sample())

0
0
1
0
0
1
0
0
1
1

In [11]:
action_space = gym.spaces.Discrete(2)
rewards = []
for i in range(50): # 1000번해도 못깸
    action = action_space.sample() # 무지한자의 행동 (찍어)
    #action = 1 # 깨달은자의 행동 (button1을 눌러)
    if action == 0: 
        reward = 1 
        rewards.append(reward)
    else:
        reward = 100
        rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100
n_try = 2   action = 1  reward = 100    rwd20 = 200
n_try = 3   action = 0  reward = 1  rwd20 = 201
n_try = 4   action = 0  reward = 1  rwd20 = 202
n_try = 5   action = 0  reward = 1  rwd20 = 203
n_try = 6   action = 1  reward = 100    rwd20 = 303
n_try = 7   action = 1  reward = 100    rwd20 = 403
n_try = 8   action = 1  reward = 100    rwd20 = 503
n_try = 9   action = 1  reward = 100    rwd20 = 603
n_try = 10  action = 0  reward = 1  rwd20 = 604
n_try = 11  action = 1  reward = 100    rwd20 = 704
n_try = 12  action = 1  reward = 100    rwd20 = 804
n_try = 13  action = 0  reward = 1  rwd20 = 805
n_try = 14  action = 0  reward = 1  rwd20 = 806
n_try = 15  action = 0  reward = 1  rwd20 = 807
n_try = 16  action = 1  reward = 100    rwd20 = 907
n_try = 17  action = 0  reward = 1  rwd20 = 908
n_try = 18  action = 1  reward = 100    rwd20 = 1008
n_try = 19  action = 0  reward = 1  rwd20 = 1009
n_try = 20  action = 0  reward = 1  rwd20 = 10

## 수정2: Env 클래스

In [12]:
class Bandit:
    def step(self,action):
        if action == 0: 
            return 1
        else:
            return 100

In [13]:
env = Bandit()
rewards = []
for i in range(50): # 1000번해도 못깸
    #action = action_space.sample() # 무지한자의 행동 (찍어)
    action = 1 # 깨달은자의 행동 (button1을 눌러)
    reward = env.step(action)
    rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100
n_try = 2   action = 1  reward = 100    rwd20 = 200
n_try = 3   action = 1  reward = 100    rwd20 = 300
n_try = 4   action = 1  reward = 100    rwd20 = 400
n_try = 5   action = 1  reward = 100    rwd20 = 500
n_try = 6   action = 1  reward = 100    rwd20 = 600
n_try = 7   action = 1  reward = 100    rwd20 = 700
n_try = 8   action = 1  reward = 100    rwd20 = 800
n_try = 9   action = 1  reward = 100    rwd20 = 900
n_try = 10  action = 1  reward = 100    rwd20 = 1000
n_try = 11  action = 1  reward = 100    rwd20 = 1100
n_try = 12  action = 1  reward = 100    rwd20 = 1200
n_try = 13  action = 1  reward = 100    rwd20 = 1300
n_try = 14  action = 1  reward = 100    rwd20 = 1400
n_try = 15  action = 1  reward = 100    rwd20 = 1500
n_try = 16  action = 1  reward = 100    rwd20 = 1600
n_try = 17  action = 1  reward = 100    rwd20 = 1700
n_try = 18  action = 1  reward = 100    rwd20 = 1800
n_try = 19  action = 1  reward = 100    rwd20 = 1900
n_

## 수정3: Agent 클래스

`-` Agent 클래스를 만들자 (액션을 하고, 환경에서 받은 reward를
간직하도록)

In [14]:
class Agent1:
    def __init__(self):
        self.action_space = gym.spaces.Discrete(2)
        self.action = None
        self.reward = None
        self.actions = []
        self.rewards = []
    def act(self):
        # self.action = self.action_space.sample() # 무지한 자
        self.action = 1 # 깨달은 자 
    def save_experience(self): 
        self.actions.append(self.action)
        self.rewards.append(self.reward)

— 대충 아래와 같은 느낌으로 돌아가요 —

**시점0**: init

In [15]:
env = Bandit()
agent = Agent1()

In [16]:
agent.action, agent.reward

**시점1**: agent \>\> env

In [17]:
agent.act()

In [18]:
agent.action, agent.reward

In [19]:
env.agent_action = agent.action 

**시점2**: agent \<\< env

In [20]:
agent.reward = env.step(env.agent_action) 
# agent.reward = env.step(agent.action) 도 같은 결과를 주는 코드임!!

In [21]:
agent.action, agent.reward, env.agent_action

In [22]:
agent.actions, agent.rewards

**시점3**: agent: `save_experience`

In [23]:
agent.save_experience()

In [24]:
agent.actions, agent.rewards

— 전체코드 —

In [25]:
env = Bandit()
agent = Agent1()
for i in range(50):
    ## 본질적인 코드
    # step1: agent >> env 
    agent.act()
    env.agent_action = agent.action 
    # step2: agnet << env 
    agent.reward = env.step(env.agent_action) 
    # step3: save 
    agent.save_experience()
    
    ## 비본질적 코드 
    print(
        f"n_try = {i+1}\t"
        f"action = {agent.action}\t"
        f"reward = {agent.reward}\t"
        f"rwd20 = {sum(agent.rewards[-20:])}"
    )
    if np.sum(agent.rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100
n_try = 2   action = 1  reward = 100    rwd20 = 200
n_try = 3   action = 1  reward = 100    rwd20 = 300
n_try = 4   action = 1  reward = 100    rwd20 = 400
n_try = 5   action = 1  reward = 100    rwd20 = 500
n_try = 6   action = 1  reward = 100    rwd20 = 600
n_try = 7   action = 1  reward = 100    rwd20 = 700
n_try = 8   action = 1  reward = 100    rwd20 = 800
n_try = 9   action = 1  reward = 100    rwd20 = 900
n_try = 10  action = 1  reward = 100    rwd20 = 1000
n_try = 11  action = 1  reward = 100    rwd20 = 1100
n_try = 12  action = 1  reward = 100    rwd20 = 1200
n_try = 13  action = 1  reward = 100    rwd20 = 1300
n_try = 14  action = 1  reward = 100    rwd20 = 1400
n_try = 15  action = 1  reward = 100    rwd20 = 1500
n_try = 16  action = 1  reward = 100    rwd20 = 1600
n_try = 17  action = 1  reward = 100    rwd20 = 1700
n_try = 18  action = 1  reward = 100    rwd20 = 1800
n_try = 19  action = 1  reward = 100    rwd20 = 1900
n_

-   위의 코드를 거의 뼈대로 사용합니당!

# Game1: 벤딧문제 – 학습과정 포함

`-` Game1에 대한 생각:

-   사실 강화학습은 “환경을 이해 $\to$ 뭘 해야 하는지 깨달음” 의
    과정에서 화살표를 수식화 하는 과정이다.
-   지금까지 살펴보면 Game1은 환경(env)을 이해하는 순간 에이전트가
    최적의 행동(action)[1]을 직관적으로 결정하였으므로 학습의 과정이
    포함되었다고 볼 수 없다.

`-` 지금까지의 코드 복습

1.  클래스를 선언하는 부분
    -   `Env`클래스의 선언
    -   `Agent`클래스의 선언
2.  환경과 에이전트를 인스턴스화 (초기화)
3.  for loop를 반복하며 게임을 진행
    -   메인코드: (1) agent \>\> env (2) agent \<\< env (3) 데이터저장
    -   비본질적코드: 학습과정 display, 학습종료조건

`-` 앞으로 구성할 코드의 형태: 에이전트가 데이터를 보고 스스로
`button1`을 눌러야 한다는 것을 추론하면 좋겠음

1.  클래스를 선언하는 부분
    -   `Env`클래스의 선언
    -   **`Agent`클래스의 선언**: **학습의 과정 포함 $\to$ act함수
        수정**
2.  환경과 에이전트를 인스턴스화 (초기화)
3.  for loop를 반복하며 게임을 진행
    -   메인코드: (1) agent \>\> env (2) agent \<\< env (3) 데이터저장
        **(4) 학습**
    -   비본질적코드: 학습과정 display, 학습종료조건

`-` 에이전트가 학습을 어떻게 하는가?[2]

-   $\pi(0) = \frac{Q_t(1)}{Q_t(1)+Q_t(0)}$
-   $\pi(1) = \frac{Q_t(0)}{Q_t(1)+Q_t(0)}$

여기에서 각각의 기호는 아래를 의미한다.

-   $\pi(0)$: 에이전트가 action = 0 을 할 확률, 즉 에이전트가
    `button0`을 누를 확률
-   $\pi(1)$: 에이전트가 action = 1 을 할 확률, 즉 에이전트가
    `button1`을 누를 확률
-   $Q_t(0)$: ($t$시점까지 파악한) action = 0 을 하였을 경우 에이전트가
    환경으로 받은 보상의 평균값
-   $Q_t(1)$: ($t$시점까지 파악한) action = 1 을 하였을 경우 에이전트가
    환경으로 받은 보상의 평균값

`-` 걱정: $t=0$이면 어쩌지? $t=1$이면 어쩌지?.. $\to$ 잡기술1: 일정
시간동안은 랜덤액션을 하면서 데이터를 쌓자.

`-` 쌓은 데이터를 바탕으로 환경을 이해하고 action을 뽑는 코드

[1] `button1`을 누른다

[2] 행동을 이런식으로 하도록 “전략(=정책)”을 설정하는 것은 상식적이다.
그렇지만 유일한 해결책은 아님!!

In [26]:
actions = np.array([0,0,1,0,0,1,1,1,1,0])
actions 

In [27]:
rewards = np.array([1,0.9,105,1.2,1,95,100,101,90,1])
rewards

In [28]:
q = np.array([rewards[actions==0].mean(),rewards[actions==1].mean()])
q

In [29]:
action = np.random.choice([0,1],p=q/q.sum())
action

`-` 코드를 구현해보자.

In [30]:
# 게임을 푸는 전략을 바꾸는 것이지 게임자체를 바꿀게 아니니까 수정할게 없죠?
class Bandit():
    def step(self,action):
        if action == 0: 
            return 1
        else:
            return 100

In [31]:
# learn 추가
class Agent1:
    def __init__(self):
        self.action_space = gym.spaces.Discrete(2)
        self.action = None
        self.reward = None
        self.actions = []
        self.rewards = []
        self.n_experiences = 0
        self.q = np.array([0.001,0.001]) # 사실 [0,0] 을 넣고싶음
    def act(self):
        self.action = np.random.choice([0, 1], p=self.q/self.q.sum())
    def save_experience(self): 
        self.actions.append(self.action)
        self.rewards.append(self.reward)
        self.n_experiences = len(self.actions)
    def learn(self):
        if self.n_experiences < 30:
            pass 
        else:
            actions = np.array(self.actions)
            rewards = np.array(self.rewards)
            self.q[0] = rewards[actions==0].mean()
            self.q[1] = rewards[actions==1].mean()

In [32]:
env = Bandit()
agent = Agent1()
for i in range(60):
    ## 본질적인 코드b
    # step1: agent >> env 
    agent.act()
    env.agent_action = agent.action 
    # step2: agnet << env 
    agent.reward = env.step(env.agent_action) 
    # step3: save 
    agent.save_experience()
    agent.learn()
    
    ## 비본질적 코드 
    print(
        f"n_try = {agent.n_experiences}\t"
        f"action = {agent.action}\t"
        f"reward = {agent.reward}\t"
        f"rwd20 = {np.sum(agent.rewards[-20:])}\t"
        f"q = {agent.q}"
    )
    if np.sum(agent.rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100 q = [0.001 0.001]
n_try = 2   action = 0  reward = 1  rwd20 = 101 q = [0.001 0.001]
n_try = 3   action = 0  reward = 1  rwd20 = 102 q = [0.001 0.001]
n_try = 4   action = 0  reward = 1  rwd20 = 103 q = [0.001 0.001]
n_try = 5   action = 0  reward = 1  rwd20 = 104 q = [0.001 0.001]
n_try = 6   action = 1  reward = 100    rwd20 = 204 q = [0.001 0.001]
n_try = 7   action = 1  reward = 100    rwd20 = 304 q = [0.001 0.001]
n_try = 8   action = 1  reward = 100    rwd20 = 404 q = [0.001 0.001]
n_try = 9   action = 0  reward = 1  rwd20 = 405 q = [0.001 0.001]
n_try = 10  action = 0  reward = 1  rwd20 = 406 q = [0.001 0.001]
n_try = 11  action = 1  reward = 100    rwd20 = 506 q = [0.001 0.001]
n_try = 12  action = 0  reward = 1  rwd20 = 507 q = [0.001 0.001]
n_try = 13  action = 0  reward = 1  rwd20 = 508 q = [0.001 0.001]
n_try = 14  action = 0  reward = 1  rwd20 = 509 q = [0.001 0.001]
n_try = 15  action = 1  reward = 100    rwd20 = 609 q = 